<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

import warnings  
warnings.filterwarnings("ignore")

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = ''
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''select COUNT(id) 
            from public.VACANCIES
'''

In [5]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''select COUNT(id) 
            from public.employers
'''

In [7]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''select COUNT(id) 
            from public.areas
'''

In [9]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''select COUNT(id) 
            from public.INDUSTRIES
'''

In [11]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df


,count
0,294


***

# выводы по предварительному анализу данных
Получили  общие показатели по базе данных,  почти 50 000 вакансий разделили между собой 23 500 работадателей, которые расположены в огрномном кол-ве регионов (1362) и кроме того в 300 сферах деятельности.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
query_4_1 = f'''select 
            COUNT(v.id),
            a.name
            from public.VACANCIES v
            JOIN public.areas a ON v.area_id=a.id
            GROUP BY a.name
            ORDER BY 1 DESC
            Limit 5
'''

In [13]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

,count,name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
query_4_2 = f'''select 
            COUNT(v.id)
            from public.VACANCIES v
            WHERE (v.salary_from is not null or v.salary_to is not null)
'''

In [15]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''select 
            ROUND(AVG(v.salary_from),0) as "Средняя нижняя граница з.п.",
            ROUND(AVG(v.salary_to),0) as "Средняя верхняя граница з.п."
            from public.VACANCIES v
'''

In [17]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

,Средняя нижняя граница з.п.,Средняя верхняя граница з.п.
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''select 
            COUNT(v.id),
            v.schedule,
            v.employment
            from public.VACANCIES v
            GROUP BY v.schedule,v.employment
            ORDER BY 1 DESC
'''

In [19]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''select 
            COUNT(v.id),
            v.experience
            from public.VACANCIES v
            GROUP BY v.experience
            ORDER BY 1
'''

In [21]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,count,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


***

# выводы по детальному анализу вакансий
1. 30% вакансий приходитя на 5 городов (Москва, Санкт - Петербург, Минск, Новосибирск, Алматы)
2. Зарплатная вилка довольно большая, около 50%.
3. 70% вакансий на полную занятость (полный день + полная занятость)
4. Меньше всего вакансий для самых опытных соискателей, можно сказать, что на рынке большая нехватка сотрудников технического уровня и менеджеров низшего звена.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''select 
            COUNT(v.id),
            e.name
            from public.VACANCIES v
            JOIN public.EMPLOYERS e ON v.EMPLOYER_id=e.id
            GROUP BY e.name
            ORDER BY 1 DESC
            Limit 5
'''

In [23]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,count,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# текст запроса
query_5_2 = f'''select 
            a.name Area_name,
            COUNT(v.id) as N_vac,
            COUNT(e.id) as N_emp
            from public.areas a
            LEFT JOIN public.EMPLOYERS e ON e.area=a.id
            LEFT JOIN public.vacancies v ON v.area_id=a.id
            GROUP BY Area_name
            ORDER BY N_vac asc, N_emp desc
            Limit 1
'''

In [25]:
# результат запроса]
df = pd.read_sql_query(query_5_2, connection)
df

,area_name,n_vac,n_emp
0,Россия,0,410


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
# текст запроса
query_5_3 = f'''select 
            COUNT(distinct a.name),
            e.name
            
            from public.vacancies v
            
            LEFT JOIN public.EMPLOYERS e ON e.id = v.employer_id
            LEFT JOIN public.areas a ON a.id=v.area_id
            GROUP BY e.name
            ORDER BY 1 desc
'''

In [27]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

,count,name
0,181,Яндекс
1,152,Ростелеком
2,116,Спецремонт
3,88,Поляков Денис Иванович
4,71,ООО ЕФИН
...,...,...
14761,1,UniSol
14762,1,UNISTORY LLC
14763,1,UNIT6
14764,1,United Distribution


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
# текст запроса
query_5_4 = f'''select 
            COUNT(e.name)
            
            from public.EMPLOYERS e
            LEFT JOIN public.EMPLOYERS_INDUSTRIES ei ON ei.employer_id = e.id
            LEFT JOIN public.INDUSTRIES i ON i.id=ei.industry_id
            WHERE i.name is NULL
'''

In [29]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
# текст запроса
query_5_5 = f'''select 
            COUNT(DISTINCT i.name),
            e.name 
            from public.EMPLOYERS e
            LEFT JOIN public.EMPLOYERS_INDUSTRIES ei ON ei.employer_id = e.id
            LEFT JOIN public.INDUSTRIES i ON i.id=ei.industry_id
            GROUP BY e.name
            HAVING COUNT(DISTINCT i.name) = 4
            ORDER BY e.name 
            OFFSET 2
            LIMIT 1
'''

In [31]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

,count,name
0,4,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
# текст запроса
query_5_6 = f'''select 
            COUNT(e.id)
            from public.EMPLOYERS e
            LEFT JOIN public.EMPLOYERS_INDUSTRIES ei ON ei.employer_id = e.id
            LEFT JOIN public.INDUSTRIES i ON i.id=ei.industry_id
            Where i.name = 'Разработка программного обеспечения'
'''

In [33]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы


response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер

#print(page.tr) # Получаем тег title, отображающийся на вкладке браузера
table = page.find("table", {"class": "sortable"})

cities = []
for row in table.find_all("tr"):
    for cell in row.find_all("td"):
        name_cell = cell.find('a')
        if name_cell:
            cities.append(name_cell.text)

In [35]:
query_5_7 = f'''with tmp as (select 
            a.name,
            count(v.id) cnt
            from public.VACANCIES v
            LEFT JOIN public.areas a ON v.area_id = a.id
            LEFT JOIN public.EMPLOYERS e ON v.EMPLOYER_id=e.id
            Where e.name = 'Яндекс'
            AND a.name IN ('%s')
            GROUP BY a.name
            ORDER BY 2 ) 
            Select * from tmp
            UNION ALL  
            Select
            'total',
            sum(cnt)
            from  tmp
''' %  "','".join(cities)


In [36]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

# выводы по анализу работодателей
1. Яндекс занимает 1 место по количеству вакансий и у него их в 4 раза больше, чем у компании на 2 месте.
2. Яндекс также имеет самый большой охват по регионам. Но Поляков Денис Ивановоч (4 место по охвату и 88 вакансий) вызывает  уважение.
3. У 33% работодателей вообще не указана сфера деятельности (есть над чем работать).
4. 15% работодателей занимаются разработкой ПО.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
# текст запроса
query_6_1 = f'''select 
            COUNT(v.id)
            from public.VACANCIES v
            Where (lower(v.name) like '%данн%'
            OR lower(v.name) like '%data%')
'''

In [38]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
# текст запроса
query_6_2 = f'''select
            count(v.id)
            from vacancies as v
            WHERE (lower(v.name) LIKE '%data science%'
            OR lower(v.name) LIKE '%data scientist%'
            OR lower(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
            OR lower(v.name) LIKE '%machine learning%'
            OR lower(v.name) LIKE '%машинн%обучен%%')
            AND (v.employment = 'Стажировка' or v.experience = 'Нет опыта' or lower(v.name) LIKE '%junior%') 
'''

In [40]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# текст запроса
query_6_3 = f'''select
            count(v.id)
            from vacancies as v
            WHERE (lower(v.name) LIKE '%data science%'
            OR lower(v.name) LIKE '%data scientist%'
            OR lower(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
            OR lower(v.name) LIKE '%machine learning%'
            OR lower(v.name) LIKE '%машинн%обучен%%')
            AND (lower(v.key_skills) LIKE '%sql%' OR lower(v.key_skills) LIKE '%postgres%')            
'''

In [42]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
# текст запроса
query_6_4 = f'''select
            count(v.id)
            from vacancies as v
            WHERE (lower(v.name) LIKE '%data science%'
            OR lower(v.name) LIKE '%data scientist%'
            OR lower(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
            OR lower(v.name) LIKE '%machine learning%'
            OR lower(v.name) LIKE '%машинн%обучен%%')
            AND (lower(v.key_skills) LIKE '%python%')            
'''

In [44]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
# текст запроса 
query_6_5 = f'''with tmp as (select 
            v.key_skills,
            length(v.key_skills) - length(REPLACE(v.key_skills, '\t','')) + 1 cnt
            from vacancies as v 
            WHERE (lower(v.name) LIKE '%data science%'
            OR lower(v.name) LIKE '%data scientist%'
            OR lower(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
            OR lower(v.name) LIKE '%machine learning%'
            OR lower(v.name) LIKE '%машинн%обучен%%') 
            AND v.key_skills IS NOT NULL
            )
            Select * from tmp
            UNION ALL  
            Select
            'Среднее кол-во навыков',
            round(AVG(cnt),2)
            from  tmp 
'''

In [46]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

,key_skills,cnt
0,Python\tSQL\tРабота в команде\tOpenCV\tML,5.00
1,Oracle Pl/SQL\tБазы данных\tMachine Learning\t...,9.00
2,Python\tOpenCV\tTensorflow\tComputer Vision\tН...,8.00
3,Python\tFlask\tLinux\tAWS\tGit,5.00
4,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,5.00
...,...,...
427,Python\tScikit-learn\tPandas\tGit\tNumpy,5.00
428,Qt\tООП\tC++\tMS Visual Studio\tРазработка ком...,6.00
429,SCALA\tSQL\tМатематический анализ\tСтатистичес...,7.00
430,Python\tLinux\tKubernetes\tAWS\tGit\tPeople Ma...,6.00


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
# текст запроса 
query_6_6 = f'''with tmp as (select 
            CASE 
                WHEN (v.salary_from IS NULL) THEN v.salary_to
                WHEN (v.salary_to IS NULL) THEN v.salary_from
                ELSE (v.salary_from + v.salary_to) / 2
            END as salary
            from vacancies as v 
            WHERE (lower(v.name) LIKE '%data science%'
            OR lower(v.name) LIKE '%data scientist%'
            OR lower(v.name) LIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
            OR lower(v.name) LIKE '%machine learning%'
            OR lower(v.name) LIKE '%машинн%обучен%%')
            AND v.experience = 'От 3 до 6 лет'
            AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
            )
            Select
            round(AVG(salary),0)
            from  tmp 
'''

In [48]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

,round
0,243115.0


***

# выводы по предметному анализу
1. 4% всех вакансий имеют отношения к данным.
2. 11% от  вакансий из пп1 имеют отношения к SQL или postgres
3. 20% всех вакансий, которые имеет отношение к данным, требуют знания Python.

# Общий вывод по проекту


Если детально анализировать БД вакансий, то можно найти текущую потребность специалистов не только по специальностям, но и по регионам. Особено наглядно это будет, если делать этот анализ не в конкретный момент времени, а с какойто переодичностью, можно будет понимать спрос на ту или иную позицию и можно прогнозировать поведение З.П.

Так же интересно посмотреть в какой сфере самая высокая возможность найти удаленную работу и за какую зарплату. А также выяснить, в какой сфере работает Поляков Денис Иванович.
 


In [49]:
# текст запроса
query_bonus1 = f'''select 
            COUNT(v.id),
            v.schedule,
            i.name,
            AVG(v.salary_from) min_salary,
            AVG(v.salary_to) max_salary
            from public.VACANCIES v
            LEFT JOIN public.EMPLOYERS_INDUSTRIES ei ON v.employer_id = ei.employer_id
            LEFT JOIN public.INDUSTRIES i ON i.id=ei.industry_id
            WHERE v.schedule = 'Удаленная работа'
            AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
            GROUP BY v.schedule,i.name
            ORDER BY 1 DESC
            Limit 10
'''

In [51]:
# результат запроса
df = pd.read_sql_query(query_bonus1, connection)
df

,count,schedule,name,min_salary,max_salary
0,2014,Удаленная работа,"Интернет-компания (поисковики, платежные систе...",46528.067806,110589.028614
1,1169,Удаленная работа,Разработка программного обеспечения,116908.095960,187118.441645
2,1008,Удаленная работа,"Системная интеграция, автоматизации технологи...",107756.114679,177567.169297
3,901,Удаленная работа,None,115601.879162,165031.568306
4,160,Удаленная работа,"Маркетинговые, рекламные, BTL, дизайнерские, E...",92020.434109,139933.028571
5,147,Удаленная работа,Кадровые агентства,175968.423077,224624.380435
6,128,Удаленная работа,Консалтинговые услуги,92341.485437,142600.428571
7,106,Удаленная работа,Интернет-провайдер,67644.720430,113106.677966
8,81,Удаленная работа,Мобильная связь,42860.467532,54286.051282
9,70,Удаленная работа,Оптоволоконная связь,54248.322581,106654.000000


#Вывод по бонусу 1
Можно сказать, что вакансии в IT-сфере спокойно переехал на удаленный режим, без потерь в зарплате. Добро пожаловать в пост-COVID-ный мир.

In [68]:
# текст запроса
query_bonus2 = f'''select 
            v.name,
            v.key_skills,
            i.name
            from public.VACANCIES v
            LEFT JOIN public.EMPLOYERS_INDUSTRIES ei ON v.employer_id = ei.employer_id
            LEFT JOIN public.EMPLOYERS e ON v.employer_id=e.id
            LEFT JOIN public.INDUSTRIES i ON i.id=ei.industry_id
            WHERE e.name = 'Поляков Денис Иванович'
            ORDER  BY 1
'''

In [69]:
# результат запроса
df = pd.read_sql_query(query_bonus2, connection)
df

,name,key_skills,name
0,Компьютерный мастер,Техники продаж\tУверенный пользователь ПК\tРем...,None
1,Компьютерный мастер,None,None
2,Компьютерный мастер,None,None
3,Компьютерный мастер,None,None
4,Компьютерный мастер,None,None
...,...,...,...
122,Ученик компьютерного мастера,None,None
123,Ученик компьютерного мастера,None,None
124,Ученик компьютерного мастера,None,None
125,Ученик компьютерного мастера,None,None


Судя по всему Поляков Д.И. владелец сети сервисных центров.